# Tutorial 2: Sigma-Delta Modulator — Design Flow with SKY130

This tutorial walks through a first-order switched-capacitor sigma-delta modulator
implemented in the SKY130 open PDK. It covers the full flow from behavioral simulation
to transistor-level layout, parasitic extraction, and digital backend integration.

Unlike Tutorial 1, which covered core `pade` features in isolation, this tutorial focuses on
the infrastructure you build *on top of* core `pade` to support a real design project.
Core `pade` imposes no project structure, naming conventions, or class hierarchies — the
organization shown here is one approach that works well for this design, not a requirement.
Use what makes sense, ignore the rest.

The tutorial is structured as a guided tour through the project directory.
Each section points you to the relevant source files, explains what they do and why,
and runs key steps inline.

## 1. Project Structure

The `examples/` directory is organized as a self-contained design project:

```
examples/
├── pdk/                        # PDK configuration (process-specific)
│   ├── rules.py                #   Generic design-rule base classes
│   └── sky130/                 #   SKY130-specific implementation
│       ├── config.py           #     Tool paths, directories
│       ├── layers.py           #     Layer definitions + GDS mapping
│       ├── rules.py            #     DRC rules (spacing, width, enclosure)
│       ├── layout.py           #     SKY130LayoutCell base class
│       ├── vias.py             #     Via definitions + layer stack
│       ├── pex.py              #     PEX netlist substitution decorator
│       └── primitives/         #     Primitive device cells
│           ├── transistors/    #       MOSFET schematic + layout
│           └── capacitors/     #       MiM cap schematic + layout
│
├── src/                        # Design source (your circuits)
│   ├── components/             #   Cell libraries
│   │   ├── behavioral/         #     Ideal models (OTA, switch, comparator)
│   │   ├── digital/            #     Standard cells + RTL
│   │   │   ├── schematic.py    #       IVX, TGX, NSAL, NSALRSTL, NSALCMP
│   │   │   ├── layout.py       #       Corresponding layouts
│   │   │   └── rtl/            #       Verilog source (CIC filter)
│   │   ├── mod1/               #     First-order modulator (transistor-level)
│   │   └── dsm_top/            #     Top-level integration (analog + digital)
│   ├── testbenches/            #   Simulation testbenches
│   └── runners/                #   Simulation runners (run + evaluate pattern)
│
├── utils/                      # EDA tool wrappers
│   ├── design_runner.py        #   Unified DRC/LVS/PEX runner
│   ├── drc.py                  #   Magic DRC wrapper
│   ├── lvs.py                  #   Netgen LVS wrapper
│   ├── pex.py                  #   Magic PEX extraction wrapper
│   ├── yosys.py                #   Yosys synthesis wrapper
│   ├── openroad.py             #   OpenROAD P&R wrapper
│   └── iverilog.py             #   Icarus Verilog simulation wrapper
│
├── scripts/                    # Standalone test/verification scripts
│   ├── test_transistor_layout.py
│   ├── test_cap_mim.py
│   ├── test_digital_layout.py
│   └── ...
│
└── tutorials/                  # These notebooks
```

### Separation of concerns

Nothing under `examples/` is part of core `pade`. The separation is intentional:

| Layer | What | Where |
|-------|------|-------|
| **Core** | `Cell`, `LayoutCell`, `Terminal`, `Net`, `route()`, `GDSWriter`, ... | `pade/` |
| **PDK** | Layers, rules, via stack, primitive device cells | `examples/pdk/sky130/` |
| **Design** | Your circuits, testbenches, simulation runners | `examples/src/` |
| **Tools** | Wrappers around Magic, Netgen, Yosys, OpenROAD, ... | `examples/utils/` |

When starting a new project on a different PDK, you could copy `examples/` and replace
the `pdk/sky130/` directory with your target process. The `src/` and `utils/` structure
carries over.

### Environment

This tutorial requires a full EDA tool environment. The project ships a
`.devcontainer/Dockerfile` based on
[IIC-OSIC-TOOLS](https://github.com/iic-jku/IIC-OSIC-TOOLS), which bundles:

- **NGspice** — circuit simulation
- **Magic** — DRC and parasitic extraction
- **Netgen** — LVS
- **KLayout** — GDS viewing and DEF→GDS conversion
- **Yosys** — RTL synthesis
- **OpenROAD** — place & route
- **Icarus Verilog** — RTL simulation
- **SKY130 PDK** — installed at `$PDKPATH`

If you are running outside the devcontainer, ensure these tools are on your `$PATH`
and the `PDK`, `PDKPATH`, and `STD_CELL_LIBRARY` environment variables are set.

In [1]:
import sys
from pathlib import Path

# Add examples/ to the Python path so we can import pdk, src, utils
EXAMPLES_DIR = Path('..').resolve()
if str(EXAMPLES_DIR) not in sys.path:
    sys.path.insert(0, str(EXAMPLES_DIR))

# Verify SKY130 PDK is accessible
from pdk.sky130.config import config

print(f'Project root: {config.project_root}')
print(f'Tech file:    {config.tech_file}  (exists: {config.tech_file.exists()})')
print(f'Liberty (tt): {config.liberty_tt.name}  (exists: {config.liberty_tt.exists()})')

Project root: /workspaces/pade/examples
Tech file:    /foss/pdks/sky130A/libs.tech/magic/sky130A.tech  (exists: True)
Liberty (tt): sky130_fd_sc_hd__tt_025C_1v80.lib  (exists: True)


---
## 2. PDK Configuration

Everything process-specific lives under `pdk/sky130/`. When porting to a different PDK,
this is the directory you replace. The rest of the project (`src/`, `utils/`) stays the same.

The key files are:

| File | Purpose |
|------|---------|
| `config.py` | Tool paths, PDK paths, project directories |
| `layers.py` | Layer objects with GDS mapping, convenience aliases |
| `rules.py` | Design rules — spacing, width, enclosure, area (all in nm) |
| `vias.py` | Via definitions, layer stack, cut-count helpers |
| `layout.py` | `SKY130LayoutCell` — PDK-aware base class for layout |
| `pex.py` | `pex_enabled` decorator for PEX netlist substitution |

### config.py

Centralizes all paths — PDK root, Magic tech file, Netgen setup, standard cell libraries,
and project output directories. A module-level singleton (`config`) is imported everywhere.

See `pdk/sky130/config.py`.

In [2]:
print(f'Project root:  {config.project_root}')
print(f'Layout dir:    {config.layout_dir}')
print(f'Work dir:      {config.work_dir}')
print(f'Sim data dir:  {config.sim_data_dir}')
print(f'Tech file:     {config.tech_file.name}')
print(f'Netgen setup:  {config.netgen_setup.name}')

Project root:  /workspaces/pade/examples
Layout dir:    /workspaces/pade/examples/layout
Work dir:      /workspaces/pade/examples/work
Sim data dir:  /workspaces/pade/examples/sim_data
Tech file:     sky130A.tech
Netgen setup:  sky130A_setup.tcl


### layers.py

Defines `Layer` objects for every SKY130 layer. A full layer map is auto-generated from
the PDK installation (for GDS I/O), and convenience constants provide short names for
common layers. The `connectivity=True` flag marks layers that carry electrical signal —
used by `check_shorts()` and routing connectivity checks.

See `pdk/sky130/layers.py`.

In [3]:
from pdk.sky130.layers import (
    sky130_layers, M1, M2, M3, M4, M5, LI, POLY, DIFF, VIA1, VIA2
)

# The full layer map (used by GDSWriter/GDSReader)
print(f'Layer map: {sky130_layers.pdk_name}, {len(sky130_layers.mapping)} layers')

# Convenience constants are plain Layer objects
print(f'\nM1 = {M1}  connectivity={M1.connectivity}')
print(f'M5 = {M5}  connectivity={M5.connectivity}')
print(f'POLY = {POLY}  connectivity={POLY.connectivity}')

Layer map: sky130A, 82 layers

M1 = MET1:drawing  connectivity=True
M5 = MET5:drawing  connectivity=True
POLY = POLY:drawing  connectivity=True


### rules.py

All design rules in one place, stored as nested `DesignRules` objects with attribute access.
Dimensions are in nm. The generic `DesignRules` class (in `pdk/rules.py`) is just a dict
wrapper — nothing SKY130-specific about the container itself.

See `pdk/sky130/rules.py`.

In [4]:
from pdk.sky130.rules import sky130_rules

# Metal rules
print(f'M1: min_width={sky130_rules.M1.MIN_W} nm, min_spacing={sky130_rules.M1.MIN_S} nm')
print(f'M5: min_width={sky130_rules.M5.MIN_W} nm, min_spacing={sky130_rules.M5.MIN_S} nm')

# Via rules
print(f'\nVIA1: cut={sky130_rules.VIA1.W} nm, spacing={sky130_rules.VIA1.S} nm')
print(f'  M1 enclosure: {sky130_rules.VIA1.ENC_BOT} nm (min), {sky130_rules.VIA1.ENC_BOT_ADJ} nm (adj side)')
print(f'  M2 enclosure: {sky130_rules.VIA1.ENC_TOP} nm (min), {sky130_rules.VIA1.ENC_TOP_ADJ} nm (adj side)')

# MOSFET rules
print(f'\nMOSFET: min_L={sky130_rules.MOS.MIN_L} nm, min_W={sky130_rules.MOS.MIN_W} nm')
print(f'  gate extension over diff: {sky130_rules.MOS.GATE_EXT} nm')

M1: min_width=140 nm, min_spacing=140 nm
M5: min_width=1600 nm, min_spacing=1600 nm

VIA1: cut=150 nm, spacing=170 nm
  M1 enclosure: 55 nm (min), 85 nm (adj side)
  M2 enclosure: 55 nm (min), 85 nm (adj side)

MOSFET: min_L=150 nm, min_W=420 nm
  gate extension over diff: 130 nm


### vias.py

Defines `ViaDefinition` dataclasses for each via level (MCON, VIA1–VIA4), constructed
from the rules in `rules.py`. Provides cut-count calculation, enclosure sizing, and
`get_via_stack()` which returns the full list of intermediate vias between any two layers
(e.g. LI → M3 gives [MCON, VIA1, VIA2]).

See `pdk/sky130/vias.py`.

In [5]:
from pdk.sky130.vias import VIA1_DEF, get_via_stack, LAYER_STACK

print('Layer stack:', ' → '.join(l.name for l in LAYER_STACK))

# Via between M1 and M2
print(f'\nVIA1: {VIA1_DEF.bot_layer.name} → {VIA1_DEF.top_layer.name}')
print(f'  cut: {VIA1_DEF.cut_w}×{VIA1_DEF.cut_h} nm, spacing: {VIA1_DEF.cut_space} nm')
print(f'  min route width for corner via: {VIA1_DEF.min_route_width()} nm')

# Multi-level via stack
stack = get_via_stack(LI, M3)
print(f'\nLI → M3 requires {len(stack)} vias: {[v.name for v in stack]}')

Layer stack: LI1 → MET1 → MET2 → MET3 → MET4 → MET5

VIA1: MET1 → MET2
  cut: 150×150 nm, spacing: 170 nm
  min route width for corner via: 580 nm

LI → M3 requires 3 vias: ['MCON', 'VIA1', 'VIA2']


### layout.py — SKY130LayoutCell

This is the PDK-aware base class for all SKY130 layout cells. It extends core
`LayoutCell` with:

- **Design rules** — `self.rules` gives attribute access to all rules from `rules.py`
- **`_get_layer_min_spacing()` / `_get_layer_min_width()`** — overrides that feed real
  rules into `route()` and `track` offset calculation
- **`add_via()`** — draws via stacks between any two routing layers, with automatic
  cut-count calculation and enclosure sizing
- **`route()`** — extended to support multi-layer routes with automatic corner vias
- **`stack_column()`** — placement helper for vertically stacking cells

Every layout cell in this project inherits from `SKY130LayoutCell` instead of the
bare `LayoutCell`.

See `pdk/sky130/layout.py`.

In [6]:
from pdk.sky130.layout import SKY130LayoutCell

# Quick demo: SKY130LayoutCell knows real design rules
demo = SKY130LayoutCell(cell_name='demo')
print(f'M1 min spacing: {demo._get_layer_min_spacing(M1, 140)} nm')
print(f'M1 min width:   {demo._get_layer_min_width(M1)} nm')

# Via between M1 and M2 at origin
shapes = demo.add_via(M1, M2, cx=0, cy=0, nx=1, ny=1, net='SIG')
print(f'\nSingle M1→M2 via: {len(shapes)} shapes (cuts + metal enclosures)')

M1 min spacing: 140 nm
M1 min width:   140 nm

Single M1→M2 via: 3 shapes (cuts + metal enclosures)


---
## 3. Primitive Devices

Primitive devices are the lowest-level cells that map directly to PDK models.
They live under `pdk/sky130/primitives/` and have both schematic and layout implementations.

### Transistor schematic

The transistor schematic is loaded directly from a SPICE subcircuit file using `load_subckt()`.
This avoids manually defining terminals and parameters — they are parsed from the `.spice` file.

See `pdk/sky130/primitives/transistors/schematic.py` (2 lines of code).

In [7]:
from pdk.sky130.primitives.transistors.schematic import Nfet01v8, Pfet01v8

nmos = Nfet01v8(instance_name='M1', w=1.0, l=0.15, nf=2)
print(f'Cell: {nmos.cell_name}')
print(f'Terminals: {list(nmos.terminals.keys())}')
print(f'Parameters: { {k: v.value for k, v in nmos.parameters.items()} }')

Cell: nfet_01v8
Terminals: ['d', 'g', 's', 'b']
Parameters: {'w': '1.0', 'l': '0.15', 'nf': '2'}


### Transistor layout

`MOSFET_Layout` is the base class that generates the full transistor geometry —
gate, source/drain contacts, poly contacts, taps, implants, wells, and M1 routing.
This is essentially a Python port of the PDK's parametric cell (typically written in Tcl),
but can be freely customized for your preferred device layout style.

`NFET_01V8_Layout` and `PFET_01V8_Layout` subclass it, setting device-specific layer
assignments (implant type, well type, default tap side).

The layout reads `w`, `l`, `nf` from the linked schematic cell. Key options:
- `tap`: substrate/well tap placement — `'left'`, `'right'`, `'both'`, or `None`
- `poly_contact`: gate contact side — follows `tap` by default

See `pdk/sky130/primitives/transistors/layout.py` (~650 lines — the largest file in the project,
since transistor layout is inherently complex).

In [8]:
from pdk.sky130.primitives.transistors.layout import NFET_01V8_Layout, PFET_01V8_Layout

nmos_layout = NFET_01V8_Layout(schematic=nmos, tap='left')
print(f'NMOS: {nmos_layout.bbox()}, {len(nmos_layout.shapes)} shapes')
print(f'Pins: {list(nmos_layout.pins.keys())}')
print(f'Refs: {list(nmos_layout.refs.keys())}')

NMOS: (-75, -75, 3120, 3375), 63 shapes
Pins: ['S', 'D', 'G', 'B']
Refs: ['S', 'D', 'G', 'B', 'S0', 'S1', 'D0', 'G0', 'G1', 'DBOT', 'DTOP', 'SBUS', 'GBUS']


NMOS layout (W=1.0, L=0.15, NF=2, tap=left) in KLayout:

![NMOS layout](images/nmos_gds.png)

### MiM capacitor

The capacitor schematic (`CapMim`) wraps a PDK subcircuit and exposes `w`, `l`, `metal`
parameters. The `@pex_enabled` decorator (covered in Section 7) adds optional PEX netlist
substitution.

The layout (`CapMimLayout`) draws the MiM plate, enclosure metals, and via array.

See `pdk/sky130/primitives/capacitors/schematic.py` and `layout.py`.

In [9]:
from pdk.sky130.primitives.capacitors.schematic import CapMim
from pdk.sky130.primitives.capacitors.layout import CapMimLayout

cap = CapMim(instance_name='C1', w=10, l=10, metal=4)
cap_layout = CapMimLayout(schematic=cap)

print(f'Cap schematic: {cap.cell_name}, terminals={list(cap.terminals.keys())}')
print(f'Cap layout: {cap_layout.bbox()}, {len(cap_layout.shapes)} shapes')
print(f'Pins: {list(cap_layout.pins.keys())}')

Cap schematic: cap_mim_m4, terminals=['TOP', 'BOT']
Cap layout: (-140, -140, 10140, 10140), 39 shapes
Pins: ['TOP', 'BOT']


### DRC and LVS

Physical verification uses Magic (DRC) and Netgen (LVS), wrapped in `utils/drc.py` and
`utils/lvs.py`. The `DesignRunner` (in `utils/design_runner.py`) provides a unified interface.

The flow is: write GDS → Magic reads it and runs DRC → Netgen compares layout netlist to schematic.

In [10]:
from pade.backends.gds.layout_writer import GDSWriter
from pdk.sky130.layers import sky130_layers
from utils.design_runner import DesignRunner

gds_writer = GDSWriter(layer_map=sky130_layers)

# Write GDS for both primitives (cap GDS needed later for PEX)
gds_writer.write(nmos_layout, config.layout_dir)
gds_writer.write(cap_layout, config.layout_dir)

# Run DRC + LVS on transistor
runner = DesignRunner(nmos_layout, nmos)
print(runner.drc())
print(runner.lvs())

DRC PASS: 0 errors

LVS:
                         #       ###################       _   _
                        #        #                 #       *   *
                   #   #         #     CORRECT     #         |
                    # #          #                 #       \___/
                     #           ###################



---
## 4. Digital Standard Cells

The sigma-delta modulator uses custom digital cells for switching and comparison.
These live in `src/components/digital/` with separate schematic and layout files.

### Cell hierarchy

| Cell | Function | Subcells |
|------|----------|----------|
| **IVX** | CMOS inverter | 1 NFET + 1 PFET |
| **TGX** | Transmission gate | 1 NFET + 1 PFET |
| **NSAL** | N-type Strong Arm latch | 12 transistors + 2 IVX output buffers |
| **NSALRSTL** | SR reset latch | 9 transistors (CLK-gated, cross-coupled) |
| **NSALCMP** | Full comparator | 1 NSAL + 1 NSALRSTL |

The comparator (`NSALCMP`) is the top-level digital block used in the modulator.

See `src/components/digital/schematic.py` and `layout.py`.

### Schematic

Each cell follows the same pattern from Tutorial 1: define terminals, set parameters,
instantiate subcells, and connect. For example, `IVX` is just two transistors:

```python
class IVX(Cell):
    def __init__(self, ..., wn=1.0, wp=2.0, l=0.15, nf=1):
        ...
        self.MP = Pfet01v8(w=wp, l=l, nf=nf)
        self.MP.connect(['d', 'g', 's', 'b'], ['OUT', 'IN', 'VDD', 'VDD'])
        self.MN = Nfet01v8(w=wn, l=l, nf=nf)
        self.MN.connect(['d', 'g', 's', 'b'], ['OUT', 'IN', 'VSS', 'VSS'])
```

The comparator (does perhaps not belong in digital library) builds up through composition: `NSALCMP` contains a `NSAL` (StrongARM latch core)
and a `NSALRSTL` (reset latch), each of which contains individual transistors and inverters.
Source: https://ieeexplore.ieee.org/document/7130773

In [11]:
from src.components.digital.schematic import IVX, TGX, NSALCMP

cmp = NSALCMP(instance_name='CMP', wn=1.0, l=0.15)
print(f'NSALCMP terminals: {list(cmp.terminals.keys())}')
print(f'Subcells: {list(cmp.subcells.keys())}')
print(f'  SAL subcells:  {list(cmp.SAL.subcells.keys())}')
print(f'  RSTL subcells: {list(cmp.RSTL.subcells.keys())}')

NSALCMP terminals: ['AVDD', 'AVSS', 'CLK', 'INP', 'INN', 'OP', 'ON']
Subcells: ['SAL', 'RSTL']
  SAL subcells:  ['MN0A', 'MN0B', 'MN1A', 'MN1B', 'MN2A', 'MN2B', 'MP1A', 'MP1B', 'MP2A', 'MP2B', 'MP3A', 'MP3B', 'I0', 'I1']
  RSTL subcells: ['MN1A', 'MN2A', 'MN3A', 'MN1B', 'MN2B', 'MN3B', 'MP1A', 'MP1B', 'MP_DUMMY']


### Layout

Each layout cell inherits from `SKY130LayoutCell` and follows a consistent structure:

1. Instantiate transistor layouts linked to their schematic counterparts
2. Place and align (typically using `stack_column` for vertical stacking)
3. Route internal connections (poly bridges, M1 drains/sources, M2 signal routing)
4. Add pins

The `IVXLayout` places NFET left and PFET right, bridges poly gates, and routes
drain (OUT) and source (VDD/VSS) on M1.

Higher-level cells compose these: `NSALCMPLayout` instantiates `NSALLayout` and
`NSALRSTLLayout`, places them side by side, and routes the interconnects on M2/M3.

See `src/components/digital/layout.py`.

In [12]:
from src.components.digital.layout import NSALCMPLayout

cmp_layout = NSALCMPLayout(schematic=cmp)
gds_writer.write(cmp_layout, config.layout_dir)

print(f'NSALCMP layout: {cmp_layout.bbox()}')
print(f'Pins: {list(cmp_layout.pins.keys())}')
print(f'Total shapes (flat): {len(cmp_layout.get_all_shapes())}')

NSALCMP layout: (-75, -180, 7940, 30430)
Pins: ['INP', 'INN', 'CLK', 'OP', 'ON', 'AVDD', 'AVSS']
Total shapes (flat): 1593


NSALCMP layout in KLayout:

![NSALCMP layout](images/NSALCMP_GDS.png)

In [13]:
# DRC + LVS on the comparator
cmp_runner = DesignRunner(cmp_layout, cmp)
print(cmp_runner.drc())
print(cmp_runner.lvs())

DRC PASS: 0 errors

LVS:
                         #       ###################       _   _
                        #        #                 #       *   *
                   #   #         #     CORRECT     #         |
                    # #          #                 #       \___/
                     #           ###################



---
## 5. Behavioral Simulation

Before investing time in transistor-level layout and verification, we validate the system architecture with fast behavioral models. The `src/components/behavioral/`
directory contains B-source models for the sigma-delta building blocks:

| Class | File | Description |
|---|---|---|
| `IdealSwitch` | `behavioral/schematic.py` | Voltage-controlled switch (smooth tanh transition) |
| `Ota` | `behavioral/schematic.py` | Single-ended OTA with inverter interface |
| `IdealComparator` | `behavioral/schematic.py` | Clocked sample-compare-latch comparator |
| `SigmaDelta1` | `behavioral/schematic.py` | Complete first-order SC modulator |

`SigmaDelta1` instantiates the same switch-capacitor topology as the schematic-level `Mod1`,
but uses `IdealSwitch` instead of `TGX`, `Ota` instead of `IVX`, and `IdealComparator`
instead of `NSALCMP`.

In [14]:
from src.components.behavioral.schematic import SigmaDelta1

# Behavioral modulator hierarchy
sd = SigmaDelta1()
print(f'{sd.cell_name}')
for name, sub in sd.subcells.items():
    print(f'  {name:6s} -> {sub.cell_name}')
    for sname, ssub in sub.subcells.items():
        print(f'           {sname:6s} -> {ssub.cell_name}')

sigma_delta_1
  Cs     -> C
  Cf     -> C
  Caz    -> C
  OTA    -> ota
           Vref   -> V
           B1     -> B
           R1     -> R
  S1     -> ideal_switch
           B1     -> B
  S2     -> ideal_switch
           B1     -> B
  S3     -> ideal_switch
           B1     -> B
  S4     -> ideal_switch
           B1     -> B
  S5     -> ideal_switch
           B1     -> B
  S6     -> ideal_switch
           B1     -> B
  S7     -> ideal_switch
           B1     -> B
  S8     -> ideal_switch
           B1     -> B
  Csc    -> C
  COMP   -> ideal_comparator
           B1     -> B
           B2     -> B
           S_latch_p -> ideal_switch
           C_latch_p -> C
           S_latch_n -> ideal_switch
           C_latch_n -> C
           B_buf_p -> B
           B_buf_n -> B


### Schematic-level modulator

The `Mod1` class in `src/components/mod1/schematic.py` replaces every behavioral block
with real PDK cells:

| Behavioral | Schematic (Mod1) |
|---|---|
| `IdealSwitch` | `TGX` (transmission gate) |
| `Ota` | `IVX` (CMOS inverter as amplifier) |
| `IdealComparator` | `NSALCMP` (Strong-Arm latch comparator) |
| `C` (ideal) | `CapMim` (MiM capacitor) |

Both classes share the same terminal interface, so the testbench works with either.

In [15]:
from src.components.mod1.schematic import Mod1

mod1 = Mod1()
print(f'{mod1.cell_name}')
for name, sub in mod1.subcells.items():
    print(f'  {name:6s} -> {sub.cell_name}')

Mod1
  Cs     -> cap_mim_m4
  Cf     -> cap_mim_m4
  Caz    -> cap_mim_m4
  IVAMP  -> inverter
  S1     -> tgate
  S2     -> tgate
  S3     -> tgate
  S4     -> tgate
  S5     -> tgate
  S6     -> tgate
  S7     -> tgate
  S8     -> tgate
  Csc    -> cap_mim_m4
  COMP   -> NSALCMP


### Testbench and runner

The testbench (`src/testbenches/sigma_delta.py`) generates supply, reference, input sine,
and non-overlapping clock stimuli. It accepts a `dut_class` argument to select which
modulator implementation to simulate:

```python
# Default: behavioral
tb = SigmaDeltaTranTB()

# Schematic-level
tb = SigmaDeltaTranTB(dut_class=Mod1)
```

The `SigmaDeltaRunner` wraps testbench construction, simulation statements, and result
parsing into a single class. It passes `dut_class` through to the testbench.

In [16]:
from src.runners.sigma_delta_runner import SigmaDeltaRunner

# Run behavioral simulation (fast: ~1 min)
runner = SigmaDeltaRunner(dut_class=SigmaDelta1, fs=128e3)
result = runner.run_and_evaluate(n_periods=128)

print(f'Time points: {len(result["time"])}')
print(f'Vin:  {result["vin"].min():.3f} .. {result["vin"].max():.3f} V')
print(f'Out:  {result["out"].min():.3f} .. {result["out"].max():.3f} V')
print(f'Vint: {result["vint"].min():.3f} .. {result["vint"].max():.3f} V')

[INFO] Netlist written to /workspaces/pade/examples/sim_data/sigma_delta/run/SigmaDeltaTranTB.spice
[INFO] Running: ngspice -b /workspaces/pade/examples/sim_data/sigma_delta/run/SigmaDeltaTranTB.spice -r /workspaces/pade/examples/sim_data/sigma_delta/run/output.raw


[INFO] NGspice complete in 43.6s


Time points: 2560
Vin:  0.800 .. 1.000 V
Out:  -0.000 .. 1.800 V
Vint: 0.427 .. 1.980 V


In [17]:
from bokeh.plotting import figure, show
from bokeh.layouts import column
from bokeh.io import output_notebook
from bokeh.models import Range1d
output_notebook()

t = result['time'] * 1e3  # ms

p1 = figure(title='Behavioral: Input & Integrator Output', width=800, height=250,
            x_axis_label='Time [ms]', y_axis_label='Voltage [V]')
p1.line(t, result['vin'], legend_label='vin', color='steelblue')
p1.line(t, result['vint'], legend_label='vint', color='orange')
p1.legend.location = 'top_right'

p2 = figure(title='Behavioral: Digital Output', width=800, height=200,
            x_axis_label='Time [ms]', y_axis_label='Voltage [V]',
            x_range=p1.x_range)
p2.line(t, result['out'], color='green')
p2.y_range = Range1d(-0.2, 2.0)

show(column(p1, p2))

Loading BokehJS ...

### Schematic-level simulation

Switching to `Mod1` runs the same testbench with transistor-level cells. This is
significantly slower because NGspice must solve the full MOSFET equations.

> **Note:** The NSALCMP comparator has a design issue — it outputs logic high on every
> clock cycle regardless of the integrator voltage. This is visible in the output waveform
> as a constant stream of 1s. The behavioral model works correctly. 

In [18]:
# Run schematic-level simulation (slower: includes PDK MOSFET models)
runner_schem = SigmaDeltaRunner(dut_class=Mod1, fs=128e3)
result_schem = runner_schem.run_and_evaluate(n_periods=128)

print(f'Time points: {len(result_schem["time"])}')
print(f'Out range:   {result_schem["out"].min():.3f} .. {result_schem["out"].max():.3f} V')

[INFO] Netlist written to /workspaces/pade/examples/sim_data/sigma_delta/run/SigmaDeltaTranTB.spice
[INFO] Running: ngspice -b /workspaces/pade/examples/sim_data/sigma_delta/run/SigmaDeltaTranTB.spice -r /workspaces/pade/examples/sim_data/sigma_delta/run/output.raw


[INFO] NGspice complete in 48.2s


Time points: 2560
Out range:   1.510 .. 1.800 V


In [19]:
t_s = result_schem['time'] * 1e3

p1 = figure(title='Schematic: Input & Integrator Output', width=800, height=250,
            x_axis_label='Time [ms]', y_axis_label='Voltage [V]')
p1.line(t_s, result_schem['vin'], legend_label='vin', color='steelblue')
p1.line(t_s, result_schem['vint'], legend_label='vint', color='orange')
p1.legend.location = 'top_right'

p2 = figure(title='Schematic: Digital Output (comparator stuck high — design issue)',
            width=800, height=200,
            x_axis_label='Time [ms]', y_axis_label='Voltage [V]',
            x_range=p1.x_range)
p2.line(t_s, result_schem['out'], color='green')
p2.y_range = Range1d(-0.2, 2.0)

show(column(p1, p2))

---
## 6. Parasitic Extraction (PEX)

Post-layout simulation accounts for parasitic capacitances and resistances introduced
by the physical layout. The flow is:

1. **Extract** — Magic reads the GDS and produces a SPICE netlist with parasitics
2. **Swap** — The `pex_enabled` decorator transparently replaces a cell's subcircuit
   with the PEX netlist when a `pex` kwarg is passed
3. **Simulate** — The same testbench and runner work unchanged

This is demonstrated on the MiM capacitor, where the parasitic fringing capacitance
adds a few percent to the nominal value.

### Extraction with DesignRunner

The `DesignRunner` wraps DRC, LVS, and PEX into a single interface. The GDS was already
written in Section 3. Here we run PEX to extract parasitics:

In [20]:
# Write cap GDS (needed for PEX extraction)
from pade.backends.gds.layout_writer import GDSWriter
from pdk.sky130.primitives.capacitors.layout import CapMimLayout
from pdk.sky130.primitives.capacitors.schematic import CapMim
from pdk.sky130.layers import sky130_layers

cap_schem = CapMim(w=10, l=10, metal=4)
cap_layout = CapMimLayout(schematic=cap_schem)

gds = GDSWriter(layer_map=sky130_layers)
gds.write(cap_layout, config.layout_dir)

# Run PEX
cap_runner = DesignRunner(cap_layout, cap_schem)
pex_result = cap_runner.pex()
pex_result

PEXResult(success=True, netlist_path='/workspaces/pade/examples/pex/cap_mim_m4_W10_L10_MULT1/pex_rc.spice')

### The `pex_enabled` decorator

The `CapMim` schematic class is decorated with `@pex_enabled` (see `pdk/sky130/pex.py`).
When a testbench passes `pex={'C1': 'rc'}`:

1. During `__init__`, the decorator stores the PEX config on the instance
2. When the parent assigns `self.C1 = CapMim(...)`, `Cell.__setattr__` sets the
   instance name and calls `_post_register()`
3. `_post_register` matches instance name `C1` against the PEX config, builds the
   parameterized cell name (`cap_mim_m4_W10_L10_MULT1`), and swaps `source_path`
   to the extracted netlist
4. Subcells and parameters are cleared (the PEX netlist replaces the full hierarchy)

The testbench and runner code stays identical — only a kwarg changes:

In [21]:
from src.runners.cap_runner import CapRunner

runner_cap = CapRunner(w=10, l=10)

# Pre-layout simulation
result_pre = runner_cap.run_and_evaluate('prelayout')
print(f'Pre-layout  C at 1 MHz: {result_pre["C_1MHz"]*1e15:.1f} fF')

# Post-layout simulation (PEX netlist swapped in via kwarg)
result_post = runner_cap.run_and_evaluate('postlayout', pex={'C1': 'rc'})
print(f'Post-layout C at 1 MHz: {result_post["C_1MHz"]*1e15:.1f} fF')

delta = (result_post['C_1MHz'] - result_pre['C_1MHz']) / result_pre['C_1MHz'] * 100
print(f'Parasitic increase: {delta:.1f}%')

[INFO] Netlist written to /workspaces/pade/examples/sim_data/prelayout/CapacitorAC.spice
[INFO] Running: ngspice -b /workspaces/pade/examples/sim_data/prelayout/CapacitorAC.spice -r /workspaces/pade/examples/sim_data/prelayout/output.raw
[INFO] NGspice complete in 41.8s
[INFO] PEX: C1 -> pex_rc.spice
[INFO] Netlist written to /workspaces/pade/examples/sim_data/postlayout/CapacitorAC.spice
[INFO] Running: ngspice -b /workspaces/pade/examples/sim_data/postlayout/CapacitorAC.spice -r /workspaces/pade/examples/sim_data/postlayout/output.raw


Pre-layout  C at 1 MHz: 206.0 fF


[INFO] NGspice complete in 42.9s


Post-layout C at 1 MHz: 214.8 fF
Parasitic increase: 4.3%


In [22]:
import numpy as np

freq_pre = result_pre['freq']
freq_post = result_post['freq']

p = figure(title='Capacitor Impedance: Pre- vs Post-Layout', width=800, height=350,
           x_axis_label='Frequency [Hz]', y_axis_label='|Z| [Ω]',
           x_axis_type='log', y_axis_type='log')
p.line(freq_pre, result_pre['Z'], legend_label='Pre-layout', color='steelblue', line_width=2)
p.line(freq_post, result_post['Z'], legend_label='Post-layout (PEX)', color='orange',
       line_width=2, line_dash='dashed')
p.legend.location = 'top_right'
show(p)

---
## 7. Digital Flow — CIC Decimation Filter

The sigma-delta modulator produces a 1-bit oversampled bitstream. A CIC (Cascaded
Integrator-Comb) decimation filter converts this to a multi-bit signal at a lower
sample rate. The CIC is implemented in Verilog and goes through a standard digital flow:

| Step | Tool | Wrapper |
|---|---|---|
| RTL simulation | Icarus Verilog | `utils/iverilog.py` |
| Waveform analysis | Python (VCD parser) | `utils/vcd.py` |
| Synthesis | Yosys | `utils/yosys.py` |
| Static timing analysis | OpenSTA (via OpenROAD) | `utils/openroad.py` |
| Place & route | OpenROAD | `utils/openroad.py` |

The RTL lives in `src/components/digital/rtl/`:
- `cic_filter.v` — parameterized CIC filter (ORDER=3, R=64, 1-bit in, 16-bit out)
- `cic_filter_tb.v` — testbench feeding constant bitstream, dumping VCD
- `cic_filter.sdc` — timing constraints (100 MHz clock)

Each utility wrapper follows the same pattern as the analog tools: a Python class that
invokes the tool, captures output, and returns a typed result object.

### RTL simulation

In [ ]:
from utils.iverilog import IcarusSimulator
from utils.vcd import parse_vcd

RTL_DIR = Path('..') / 'src' / 'components' / 'digital' / 'rtl'

sim = IcarusSimulator()
sim_result = sim.simulate(
    verilog_files=[RTL_DIR / 'cic_filter.v', RTL_DIR / 'cic_filter_tb.v'],
    top_module='cic_filter_tb',
    output_dir=Path('..') / 'work' / 'cic_sim',
)
sim_result

### Waveform analysis

In [ ]:
data = parse_vcd(sim_result.vcd_path)

dout = data['cic_filter_tb.dut.dout']
valid = data['cic_filter_tb.dut.valid']

t_dout = dout['times'] / 1e6  # ps -> us
t_valid = valid['times'] / 1e6

p1 = figure(title='CIC Filter Output (dout)', x_axis_label='Time [µs]',
            y_axis_label='dout', width=800, height=300)
p1.step(t_dout, dout['values'], line_width=1.5, mode='after')

p2 = figure(title='Valid Strobe', x_axis_label='Time [µs]',
            y_axis_label='valid', width=800, height=200, x_range=p1.x_range)
p2.step(t_valid, valid['values'], line_width=1.5, mode='after')

show(column(p1, p2))

### Synthesis

In [ ]:
from utils.yosys import YosysSynthesizer

synth = YosysSynthesizer()
synth_result = synth.synthesize(
    verilog_files=[RTL_DIR / 'cic_filter.v'],
    top_module='cic_filter',
    output_dir=Path('..') / 'work' / 'cic_synth',
)
synth_result

### Place & route

Full P&R flow: floorplanning, placement, clock tree synthesis, routing.
Produces a DEF layout and a GDS file for the CIC filter block.

In [ ]:
from utils.openroad import OpenROADRunner

SDC_FILE = RTL_DIR / 'cic_filter.sdc'

pnr_runner = OpenROADRunner()
pnr_result = pnr_runner.place_and_route(
    netlist=synth_result.netlist_path,
    sdc=SDC_FILE,
    top_module='cic_filter',
    output_dir=Path('..') / 'work' / 'cic_pnr',
)
pnr_result

In [ ]:
if pnr_result.sta:
    print('Post-route timing:')
    print(pnr_result.sta)

print(f'\nCIC GDS: {pnr_result.gds_path}')